In [ ]:
import glob
from importlib import import_module
from ape_producer.parse_report import make_cov_objects, square_array
import numpy as np
import pandas as pd
import re

In [ ]:
metadata = {
    "name":"SM_10_6_01"
}

In [ ]:
import data.SM_10_6_01_report_0 as report

In [ ]:
report

In [ ]:
files = glob.glob('data/{name}*.py'.format(**metadata))
modules = [f.replace('/', '.').replace('.py','') for f in files]
modules

In [ ]:
wheel_station = [[-2,  1],
       [-2,  2],
       [-2,  3],
       [-2,  4],
       [-1,  1],
       [-1,  2],
       [-1,  3],
       [-1,  4],
       [ 0,  1],
       [ 0,  2],
       [ 0,  3],
       [ 0,  4],
       [ 1,  1],
       [ 1,  2],
       [ 1,  3],
       [ 1,  4],
       [ 2,  1],
       [ 2,  2],
       [ 2,  3],
       [ 2,  4]]
alignables = ['x','y','z','phix', 'phiy', 'phiz']


In [ ]:
def sel_wheel_station(df, wheel, station):
    return df[(df.wheel==wheel) &  (df.station==station)]

In [ ]:
def get_report(module_name):
    return import_module(module_name).reports

In [ ]:
def make_hesse_df_dt(df):
    hesse_list = []
    for wheel, station in wheel_station:
        tdf = sel_wheel_station(df,wheel,station)
        covMatrix = tdf.covMatrix.mean()
        hess_stds = np.diagonal(covMatrix)**.5
        nMuons, nMuonsStD = tdf.nMuons.mean(), tdf.nMuons.std()
        hesse_list.append({"wheel": wheel, "station": station,"nMuons_mean": nMuons, "nMuons_std":nMuonsStD, **{'hesse_unc_'+dim:hess for hess, dim in zip(hess_stds, alignables)}})
    hesse_df = pd.DataFrame(hesse_list)
    return hesse_df

In [ ]:
def make_summary_report_df(module_name, njobs):
    report_ex = get_report(module_name)
    dt_cov_df, csc_cov_df = make_cov_objects(report_ex, debug=False)
    df = make_hesse_df_dt(dt_cov_df)
    df['njobs']= njobs
    return df

In [ ]:
def number_from_name(name):
    return int(re.findall('report_(\d+)', name)[0])

In [ ]:
stats_df = pd.DataFrame()
for module in modules:
    print(module)
    df = make_summary_report_df(module, number_from_name(module))
    stats_df = stats_df.append(df)

In [ ]:
stats_df.sort_values(['wheel', 'station', 'njobs'])

In [ ]:
stats_df.to_csv('data/nmuon_hesse_df.csv')